<center><img src="./images/analysis.png" height="600" width="600"/></center><br>

## Find hospitals closest to an incident

The `network` module of the ArcGIS API for Python can be used to solve different types of network analysis operations.

## Closest facility

The closest facility solver provides functionality for finding out the closest locations to a particular input point. This solver would be useful in cases when you have an incident and need to find the closest facility or need to get information on the travel time and the distance to each of the facilities from an incident point for reporting purposes.

![](http://desktop.arcgis.com/en/arcmap/latest/extensions/network-analyst/GUID-96C273DB-6A24-4D42-AADA-975A33B44F3D-web.png)

When finding closest facilities, you can specify how many to find and whether the direction of travel is toward or away from them. The closest facility solver displays the best routes between incidents and facilities, reports their travel costs, and returns driving directions.

### Connect to your GIS

In [1]:
from IPython.display import HTML
import pandas as pd
from arcgis.gis import GIS
from getpass import getpass

In [3]:
password = getpass()
url = 'https://www.arcgis.com/'
username = ''
my_gis = GIS(url, username, password)

········


### Create a Network Layer
To perform any network analysis (such as finding the closest facility, the best route between multiple stops, or service area around a facility), you would need to create a `NetworkLayer` object. We need to create a `ClosestFacilityLayer` which is a type of `NetworkLayer`.

To create any `NetworkLayer` object, you would need to provide the URL to the appropriate network analysis service. We provide a `ClosestFacility` URL to create a `ClosestFacilityLayer` object. 

In [4]:
import arcgis.network as network

In [5]:
analysis_url = my_gis.properties.helperServices.closestFacility.url
analysis_url

'https://route.arcgis.com/arcgis/rest/services/World/ClosestFacility/NAServer/ClosestFacility_World'

Create a `ClosestFacilityLayer` object using this URL

In [6]:
cf_layer = network.ClosestFacilityLayer(analysis_url, gis=my_gis)

### Create hospitals layer
Next, we will be looking for the closest hospital (facility) to an incident location. Even though we are interested in finding out the closest one, it would still be helpful to get the information on the distance and travel time to all of them for reference purposes.

We need to geocode the hospitals' addresses as well as do the reverse geocode for the incident location which has been supplied in the latitude/longitude format.

To perform the geocode operations, we import the `geocoding` module of the ArcGIS API.

In [7]:
from arcgis import geocoding

Let's geocode addresses of hospitals to create the facility layer. 

In [8]:
hospitals_addresses = ['Estrada Adhemar Bebiano, 339 Del Castilho, Rio de Janeiro RJ, 21051-370, Brazil',
                       'R. José dos Reis Engenho de Dentro, Rio de Janeiro RJ, 20750-000, Brazil',
                       'R. Dezessete, s/n Maré, Rio de Janeiro RJ, 21042-010, Brazil',
                       'Rua Dr. Miguel Vieira Ferreira, 266 Ramos, Rio de Janeiro RJ, Brazil']

Loop through each address and geocode it. The geocode operation returns a list of matches for each address. We pick the first result and extract the coordinates from it and construct a `Feature` object out of it. Then we combine all the `Feature`s representing the hospitals into a `FeatureSet` object.

In [9]:
from arcgis.features import Feature, FeatureSet

In [10]:
hosp_feat_list = []

for address in hospitals_addresses:
    hit = geocoding.geocode(address)[0]
    hosp_feat = Feature(geometry=hit['location'], attributes=hit['attributes'])

    hosp_feat_list.append(hosp_feat)

Construct a `FeatureSet` using each hospital `Feature`.

In [11]:
hospitals_fset = FeatureSet(features=hosp_feat_list, 
                            geometry_type='esriGeometryPoint', 
                            spatial_reference={'latestWkid': 4326})

In [12]:
map1 = my_gis.map('Rio de Janeiro, Brazil')
map1

MapView(layout=Layout(height='400px', width='100%'))

In [13]:
map1.draw(hospitals_fset, symbol={"type": "esriSMS","style": "esriSMSSquare",
                                  "color": [76,115,0,255],"size": 8,})

### Create incidents layer

In [14]:
incident_coords = '-43.281206,-22.865676'
reverse_geocode = geocoding.reverse_geocode({"x": incident_coords.split(',')[0], 
                                              "y": incident_coords.split(',')[1]})

incident_feature = Feature(geometry=reverse_geocode['location'], 
                           attributes=reverse_geocode['address'])

In [15]:
incident_fset = FeatureSet([incident_feature], geometry_type='esriGeometryPoint',
                          spatial_reference={'latestWkid': 4326})

In [16]:
map1.draw(incident_fset, symbol={"type": "esriSMS","style": "esriSMSCircle","size": 8})

## Solve for closest hospital
By default the closest facility service would return only the closest location, so we need to specify explicitly the `default_target_facility_count` parameter as well as `return_facilities`.


In [17]:
result = cf_layer.solve_closest_facility(incidents=incident_fset,
                                        facilities=hospitals_fset,
                                        default_target_facility_count=4,
                                        return_facilities=True,
                                        impedance_attribute_name='TravelTime',
                                        accumulate_attribute_names=['Kilometers','TravelTime'])

In [18]:
result.keys()

dict_keys(['messages', 'routes', 'facilities'])

Let us use the `routes` dictionary to construct line features out of the routes to display on the map

In [19]:
result['routes'].keys()

dict_keys(['fieldAliases', 'geometryType', 'spatialReference', 'features'])

In [20]:
result['routes']['features'][0].keys()

dict_keys(['attributes', 'geometry'])

Construct line features out of the routes that are returned.

In [21]:
line_feat_list = []
for line_dict in result['routes']['features']:
    f1 = Feature(line_dict['geometry'], line_dict['attributes'])
    line_feat_list.append(f1)

In [22]:
routes_fset = FeatureSet(line_feat_list, 
                         geometry_type=result['routes']['geometryType'],
                         spatial_reference= result['routes']['spatialReference'])

In [23]:
map1.draw(routes_fset)

Since we parsed the routes as a `FeatureSet`, we can display the attributes easily as a `pandas` `DataFrame`.

In [24]:
df1 = routes_fset.sdf
df1

,FacilityCurbApproach,FacilityID,FacilityRank,IncidentCurbApproach,IncidentID,Name,ObjectID,SHAPE,Shape_Length,Total_Kilometers,Total_Miles,Total_TravelTime
0,1,1,1,2,1,Rua Tangapeme 36 - Location 1,1,"{""paths"": [[[-43.28124189499994, -22.865707432...",0.028461,3.016318,1.874253,7.554929
1,2,4,2,2,1,Rua Tangapeme 36 - Location 4,2,"{""paths"": [[[-43.28124189499994, -22.865707432...",0.048805,5.222468,3.245091,12.202391
2,2,3,3,2,1,Rua Tangapeme 36 - Location 3,3,"{""paths"": [[[-43.28124189499994, -22.865707432...",0.061555,6.464249,4.016698,14.337250
3,2,2,4,2,1,Rua Tangapeme 36 - Location 2,4,"{""paths"": [[[-43.28124189499994, -22.865707432...",0.086735,9.143542,5.681533,18.793156


In [25]:
df1['facility_address'] = hospitals_addresses
df1['incident_address'] = [incident_feature.attributes['Match_addr'] for i in range(len(hospitals_addresses))]

In [26]:
df1[['facility_address','incident_address','Total_Miles','Total_TravelTime']]

,facility_address,incident_address,Total_Miles,Total_TravelTime
0,"Estrada Adhemar Bebiano, 339 Del Castilho, Rio...","Rua Tangapeme 2-50, Complexo do Alemão, Rio de...",1.874253,7.554929
1,"R. José dos Reis Engenho de Dentro, Rio de Jan...","Rua Tangapeme 2-50, Complexo do Alemão, Rio de...",3.245091,12.202391
2,"R. Dezessete, s/n Maré, Rio de Janeiro RJ, 210...","Rua Tangapeme 2-50, Complexo do Alemão, Rio de...",4.016698,14.337250
3,"Rua Dr. Miguel Vieira Ferreira, 266 Ramos, Rio...","Rua Tangapeme 2-50, Complexo do Alemão, Rio de...",5.681533,18.793156
